In [1]:
###################################################################################################
# 计算图
import tensorflow as tf

a = tf.constant([1.0, 2.0], name = "a")
b = tf.constant([2.0, 3.0], name = "b")

result = a + b

# 通过张量a.graph 可以查看张量所属的计算图。
# 因为没有指定，所以这个计算图应该等于当前默认的计算图
print (a.graph)
print tf.get_default_graph()
print b.graph
print result.graph

print a.graph is tf.get_default_graph()


True


In [12]:
#tensorflow 只用tf.Graph生成新的计算图，不同的计算图上的张量和运算都不会共享

import tensorflow as tf

# 在计算图g1种定义变量“v”， 并设置初始值为0
g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable("v", initializer=tf.zeros_initializer(shape=[1]))

# 在计算图g2种定义变量“v”， 并设置初始值为1
g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable("v", initializer=tf.ones_initializer(shape=[1]))
    
# 在计算图g1种读取变量“v”的值
with tf.Session(graph=g1) as sess:
    tf.initialize_all_variables().run()
    with tf.variable_scope("", reuse=True):
        print sess.run(tf.get_variable("v"))
        

# 在计算图g2种读取变量“v”的值
with tf.Session(graph=g2) as sess:
    tf.initialize_all_variables().run()
    with tf.variable_scope("", reuse=True):
        print sess.run(tf.get_variable("v"))

[ 0.]
[ 1.]


In [2]:
###################################################################################################
# tensorflow会话
# Session
import tensorflow as tf

a = tf.constant([1.0, 2.0], name = "a")
b = tf.constant([2.0, 3.0], name = "b")

result = a + b

with tf.Session() as sess:
    # 默认的会话被指定后，可以通过tf.Tensor.eval函数来计算一个张量的取值
    with sess.as_default():
        print result.eval()

# 下面两个命令有相同的功能
sess = tf.Session()
print sess.run(result)
print result.eval(session=sess)
sess.close()

print "-----------------------------------------------------"
###################################################################################################
# 通过tf.InteractiveSession函数可以省去将产生的会话注册为默认会话的过程,不能使用with

sess = tf.InteractiveSession()
print result.eval()
sess.close()

with tf.InteractiveSession() as sess:
    print result.eval()


[ 3.  5.]
[ 3.  5.]
[ 3.  5.]
-----------------------------------------------------
[ 3.  5.]


AttributeError: __exit__

In [26]:
# 通过ConfigProto配置会话
"""
    allow_soft_placement: GPU 运算可以放到CPU上运行，满足下面条件之一
        1. 运算无法在GPU上运行
        2. 没有GPU资源
        3. 运算输入包含对CPU计算结果的引用
    log_device_placement: 为True时，会记录每个节点被安排在哪个设备上
        
"""
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
sess1 = tf.InteractiveSession(config=config)
print result.eval()
sess1.close()

with tf.Session(config=config) as sess:
    print sess.run(result)
    



[ 3.  5.]
[ 3.  5.]


In [48]:
# tensorflow使用api
"""
    1. TensorFlow中维护的集合列表
        tf.GraphKeys.VARIABLES     所有变量
        tf.GraphKeys.TRAINABLE_VARIABLES     可学习变量（神经网络中的参数）
        tf.GraphKeys.SUMMARIES          日志生成的相关张量
        tf.GraphKeys.QUEUE_RUNNERS       处理输入的QueueRunner
        tf.GraphKeys.MOVING_AVERAGE_VARIABLES      所有计算滑动平均值的变量
        
    2. TensorFlow随机数生成函数
        tf.random_normal  正太分布
        tf.truncated_normal  正太分布（如果随机出来的值偏离平均值超过2个标准差，那么这个值会重新被随机）
        tf.random_uniform  平均分布
        tf.random_gamma     Gamma分布
    3. TensorFlow生成函数
        tf.zeros    产生全0数组
        tf.ones     产生全1数组
        tf.fill     产生一个全部为给定数字的数组
        tf.constant 产生一个给定值的常量
"""


# 通过变量实现神经网络参数和前向传播过程

import tensorflow as tf

w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

# 暂时将输入的特征向量定义为一个常量
x = tf.constant([[0.7, 0.9]])

# 通过前向传播算法获得神经网络的输出
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

# 初始化w1，w2，并运行
# 或者通过tf.initialize_all_variables()
sess = tf.Session()
sess.run(w1.initializer)
sess.run(w2.initializer)
print sess.run(y)
sess.close()


[[ 3.95757794]]


In [60]:
# 神经网络完整实例1
"""
"""
import tensorflow as tf
from numpy.random import RandomState

###################################################################################################
# 定义神经网络的参数，以及前向，反向传播算法

# 定义训练数据的batch大小
batch_size = 8

# 定义神经网络参数
w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

# 定义输入数据集的placeholder
# 在shape的一个维度上使用None，可以方便使用不大的batch大小，只适用于数据集比较小时
x = tf.placeholder(tf.float32, shape=(None, 2), name="x-input")
y_ = tf.placeholder(tf.float32, shape=(None, 1), name="y-input")

# 定义神经网络前向传播过程
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

# 定义损失函数和反向传播算法
cross_entropy = - tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)))
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)


###################################################################################################
# 通过RandomState随机数生成一个模拟数据集
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
# 定义规则给出样本标签。 x1 + x2 < 1 为正样本，否则为服样本
Y = [[int (x1 + x2 < 1)] for (x1, x2) in X]


###################################################################################################
# TensorFlow训练过程
with tf.Session() as sess:
    # 初始化所以变量
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    
    # 打印神经网络训练之前网络参数的值
    print '打印神经网络训练之前网络参数的值'
    print sess.run(w1)
    print sess.run(w2)
    """
        [[-0.81131822  1.48459876  0.06532937]
         [-2.44270396  0.0992484   0.59122431]]
        [[-0.81131822]
         [ 1.48459876]
         [ 0.06532937]]
    """
    
    # 设定训练的轮数
    STEPS = 5000
    for i in range(STEPS):
        # 每次选取batch_size个样本进行训练
        start = (i * batch_size) % dataset_size
        end = min (start + batch_size, dataset_size)
        
        # 通过选取的样本训练神经网络并更新参数
        sess.run(train_step, feed_dict={x: X[start:end], y_: Y[start:end]})
        
        # 每隔一段时间计算在所有数据上的交叉商并输出
        if i % 100 == 0:
            total_cross_entropy = sess.run(cross_entropy, feed_dict={x: X, y_: Y})
            print ("After %d training step, cross entropy on all data is %g" % (i, total_cross_entropy))
            
    
    # 打印训练后的神经网络参数值
    print '打印训练后的神经网络参数值'
    print sess.run(w1)
    print sess.run(w2)
    """
        [[-1.9618274   2.58235407  1.68203783]
         [-3.4681716   1.06982327  2.11788988]]
        [[-1.8247149 ]
         [ 2.68546653]
         [ 1.41819501]]
    """

打印神经网络训练之前网络参数的值
[[-0.81131822  1.48459876  0.06532937]
 [-2.44270396  0.0992484   0.59122431]]
[[-0.81131822]
 [ 1.48459876]
 [ 0.06532937]]
After 0 training step, cross entropy on all data is 0.0674925
After 100 training step, cross entropy on all data is 0.0535382
After 200 training step, cross entropy on all data is 0.0432842
After 300 training step, cross entropy on all data is 0.0364409
After 400 training step, cross entropy on all data is 0.0310934
After 500 training step, cross entropy on all data is 0.0271219
After 600 training step, cross entropy on all data is 0.0244868
After 700 training step, cross entropy on all data is 0.0224556
After 800 training step, cross entropy on all data is 0.0204244
After 900 training step, cross entropy on all data is 0.0183805
After 1000 training step, cross entropy on all data is 0.0163385
After 1100 training step, cross entropy on all data is 0.0147953
After 1200 training step, cross entropy on all data is 0.0139216
After 1300 training step,